# Projeto 1 - Ciência dos Dados

Nome: _Marco Tulio Masselli Rainho Teixeira_

Nome: _Talissa Gonçalves Albertini_

___
## Classificador automático de sentimento


Este classificador automático de sentimento tem o objetivo de promover uma otimização no processo de interpretação de feed back de qualquer produto no mercado, identificando características que os usuários aprovam ou reprovam, e no ambiente do Twitter, selecionando comentários que tenham uma maior probabilidade de serem relevantes à equipe que os analizará.

### 1. Preparação do ambiente do Jupyter

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
from string import punctuation
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('rslp')

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

### 2. Carregando a base de dados com os tweets 

In [ ]:
xls = pd.ExcelFile('xiaomi.xlsx')
df1 = pd.read_excel(xls, 'Treinamento')
df2 = pd.read_excel(xls, 'Teste')

print(df1.head())
print(df2.head())

___
### 3. Tratamento do banco de dados de treinamento

#### 3.1 Remoção de pontuação e uniformização de minúsculas

In [ ]:
def  Cleanup(df, text_column):
    df[text_column] = df[text_column].str.lower()
    df[text_column] = df[text_column].str.replace(rf'[{punctuation}]', '')
    return df

print(Cleanup(df1, 'Treinamento').head(10))
print(Cleanup(df2, 'Teste').head(10))

#### 3.2 Divisão por relevância

In [ ]:
relevant_comments = df1[df1.Relevancia == 1] 
relevant_comments_column = relevant_comments['Treinamento']
relevant_comments_list = relevant_comments_column.values.tolist()
print(relevant_comments_list[0:10])

irrelevant_comments = df1[df1.Relevancia == 0]
irrelevant_comments_column = irrelevant_comments['Treinamento']
irrelevant_comments_list = irrelevant_comments_column.values.tolist()
#print(irrelevant_comments_list)

#### 3.3 Separação de palávras 

In [ ]:
relevant_words_raw = [word for comment in relevant_comments_list for word in comment.split()]
print(relevant_words_raw[0:30])

irrelevant_words_raw = [word for comment in irrelevant_comments_list for word in comment.split()]
#print(irrelevant_words_raw)

#### 3.4 Remoção de stopwords e stemming


In [ ]:
def RemoveStopWords(list_of_words):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    filtered = []
    for word in list_of_words:
        if word not in stopwords:
            filtered.append(word)
    return filtered

stopwords = nltk.corpus.stopwords.words('portuguese')

def Stemming(list_of_words):
    stemmer = RSLPStemmer()
    filtered = []
    for word in list_of_words:
        filtered.append(stemmer.stem(word.lower()))
    return filtered

relevant_words = Stemming(RemoveStopWords(relevant_words_raw))
irrelevant_words = Stemming(RemoveStopWords(irrelevant_words_raw))


#print(relevant_words)

#### 3.5 Cálculo das frequências relativas

In [ ]:
relevant_words_serie = pd.Series(relevant_words)
relevant_words_relative = relevant_words_serie.value_counts(True)
print(relevant_words_relative.head())

irrelevant_words_serie = pd.Series(irrelevant_words)
irrelevant_words_relative = irrelevant_words_serie.value_counts(True)
#print(irrelevant_words_relative.head())

#### 3.6 Espaço amostral

In [ ]:
# Lista de todas as palávras
all_words = relevant_words + irrelevant_words
all_words[0:10]

# Frequências relativas do total de palávras
all_words_serie = pd.Series(all_words)
all_words_relative = all_words_serie.value_counts(True)
print(all_words_relative.head())

#### 3.7 Descrição das probabilidades

#### Eventos

* R: Comentário relevante
* Rc: Comentário irrelevante
* C: Comentário

#### Probabilidades

* P_R = Probabilidade do comentário ser relevante
* P_Rc = Probabilidade do comentario ser irrelevante
* P_C = Probabilidade do comentário ocorrer na lingua portugueasa = propabilidade de cada palavra ocorrer na lingua portuguesa, multiplicadas entre si
* P_C_dado_R = Probabilidade do comentário ocorrer dado que o é relevante = probabilidade de cada palavra ocorrer dado que o comentário é relevante, multiplicados entre si
* P_C_dado_Rc = Probabilidade do comentário ocorrer dado que é irrelevante = probabilidade de cada palavra ocorrer dado que o comentário é irrelevante, multiplicados entre si
* P_R_dado_C = Probabilidade do cometário ser relevante dado o comentário
* P_Rc_dado_C =Probabilidade do cometário ser irrelevante dado o comentário

### 4. Funções auxiliares

#### 4.1 Função para a suavização de Laplace 

In [ ]:
def Laplace(N_words_X, N_dif_words_X, N_dif_words_Xc):
    correction = 1/(len(N_words_X) + len(N_dif_words_X) + len(N_dif_words_Xc))
    return correction

#### 4.2 Função multiplicadora

In [ ]:
def MultiplyList(List) : 
    result = 1
    for x in List: 
         result = result * x  
    return result  

In [ ]:
df2_range = []
for row in df2.index: 
    df2_range.append(row)


### 5. Função classificador

In [ ]:
df2['test_result'] = 'NaN'

In [ ]:
def Classifier(df):
    for i in df2_range:
        comment = df2.iloc[i, 1]


        # Frequencia das palavras do comentario
        comment_words = comment.split()
        comment_words_list = Stemming(RemoveStopWords(comment_words))
        comment_words_serie = pd.Series(comment_words_list)
        comment_words_frequency = comment_words_serie.value_counts()


        P_C_dado_R = 1
        Ps_C_dado_R = []
        for j in range(len(comment_words_list)):
            if comment_words_list[j] not in relevant_words:
                Ps_C_dado_R.append(Laplace(relevant_words, relevant_words_relative, irrelevant_words_relative))
            if comment_words_list[j] == 'xiaomi':
                continue
            else:
                Ps_C_dado_R.append(relevant_words_relative[j])
        P_C_dado_R = MultiplyList(Ps_C_dado_R)
                


        P_C_dado_Rc = 1
        Ps_C_dado_Rc = []
        for j in range(len(comment_words_list)):
            if comment_words_list[j] not in irrelevant_words:
                Ps_C_dado_Rc.append(Laplace(irrelevant_words, irrelevant_words_relative, relevant_words_relative))
            if comment_words_list[j] == 'xiaomi':
                continue
            else:
                Ps_C_dado_Rc.append(irrelevant_words_relative[j])
        P_C_dado_Rc = MultiplyList(Ps_C_dado_Rc)
        

        P_C = 1
        for x in range(len(comment_words_list)):
            if comment_words_list[x] not in all_words_relative:
                continue
            else:
                P_C *= all_words_relative[comment_words_list[x]]


        P_R = len(relevant_words) / (len(relevant_words) + len(irrelevant_words))

        P_Rc = len(irrelevant_words) / (len(relevant_words) + len(irrelevant_words))

        P_R_dado_C = P_R * P_C_dado_R

        P_Rc_dado_C = P_Rc * P_C_dado_Rc


        if P_R_dado_C > P_Rc_dado_C:
            result = 1
        else:
            result = 0

        df2.iloc[i, 2] = result
    return df